In [ ]:
## Init
%matplotlib inline
import matplotlib.pyplot as plt

import urllib.request
import io
from PIL import Image

import keras.backend as K
from keras.models import load_model
import numpy as np

def load_image_online(filename):
    with urllib.request.urlopen(filename) as url:
        f = io.BytesIO(url.read())
    image = Image.open(f).convert("RGB")
    return image

def load_image(fn, image_size):
    im = load_image_online(fn)
    if (im.size[0] >= im.size[1]):
        im = im.crop(((im.size[0] - im.size[1])//2, 0, (im.size[0] + im.size[1])//2, im.size[1]))
    else:
        im = im.crop((0, (im.size[1] - im.size[0])//2, im.size[0], (im.size[0] + im.size[1])//2))
    im = im.resize((image_size, image_size), Image.BILINEAR)
    arr = np.array(im)/255*2-1   
    return arr

def arr2image(X):
    int_X = ((X+1)/2*255).clip(0,255).astype('uint8')
    return Image.fromarray(int_X)

def generate(img, fn):
    r = fn([np.array([img])])[0]
    return arr2image(np.array(r[0]))

last_load_model_path = None

In [ ]:
PLACEHOLDER_IMAGE_SIZE = 256
PLACEHOLDER_IMAGE_PATH = "http://apex.sjtu.edu.cn/public/files/avatar/20180912/lyk.jpg"
PLACEHOLDER_MODEL_PATH = "../models/VanGogh-GA.h5"

In [ ]:
## Run

if last_load_model_path != PLACEHOLDER_MODEL_PATH:
    model = load_model(PLACEHOLDER_MODEL_PATH)
    last_load_model_path = PLACEHOLDER_MODEL_PATH
    
fn_generate = K.function([model.inputs[0]],[model.outputs[0]])
input_img = load_image(PLACEHOLDER_IMAGE_PATH, PLACEHOLDER_IMAGE_SIZE)
out = generate(input_img, fn_generate)

plt.imshow(out)
plt.axis("off")
plt.show()